In [1]:
import seed_disperser

length = 50
width = 30

seeds_stocks = {
'canopy' : 500,
'tree_stratum' : 1400,
'understorey' : 1300,
'shrub_layer' : 1300
}

fig, ax = seed_disperser.disperse_seeds(seeds_stocks, length, width)

area_size : 1500
seeds_count : 4500


In [2]:
import seed_disperser

length = 100
width = 60

seeds_stocks = {
'canopy' : 2000,
'tree_stratum' : 5600,
'understorey' : 5200,
'shrub_layer' : 5200
}

fig, ax = seed_disperser.disperse_seeds(seeds_stocks, length, width)

area_size : 6000
seeds_count : 18000


In [3]:
# fig.savefig("test-png-plot.png", format='png', dpi=1000)

In [4]:
import io

imgdata = io.StringIO()
fig.savefig("test-png-plot.png", format='png', dpi=max(length,width)*5)
fig.savefig("test-png-plot.svg", format='svg')
fig.savefig(imgdata, format='svg')
imgdata.seek(0)  # rewind the data

svg_data = imgdata.getvalue()  # this is svg data
print(type(imgdata))
imgdata.close()

<class '_io.StringIO'>


In [5]:
import time, gzip

start = time.time()
svg_data = seed_disperser.get_svg(fig)
end = time.time()
print("start -> end :")
print(end - start)

start -> end :
1.4966368675231934


In [6]:
import time, gzip

start = time.time()
svg_data = seed_disperser.get_svgz(fig)
end = time.time()
print("start -> end :")
print(end - start)

start -> end :
1.5874438285827637


In [7]:
# imgdata = io.StringIO()
# fig.savefig(imgdata, format='svg')
# fig.savefig("test-png-plot.png", format='png', dpi=1000)
# imgdata.seek(0)  # rewind the data

# svg_data = imgdata.getvalue()  # this is svg data

# #save file 
# f= open("test-svg-plot.svg","w")
# f.write(svg_data)
# f.close() 
# #file('test.htm', 'w').write(svg_dta)

In [8]:
import matplotlib.pyplot as plt
plt.gcf().canvas.get_supported_filetypes()

{'eps': 'Encapsulated Postscript',
 'jpeg': 'Joint Photographic Experts Group',
 'jpg': 'Joint Photographic Experts Group',
 'pdf': 'Portable Document Format',
 'pgf': 'PGF code for LaTeX',
 'png': 'Portable Network Graphics',
 'ps': 'Postscript',
 'raw': 'Raw RGBA bitmap',
 'rgba': 'Raw RGBA bitmap',
 'svg': 'Scalable Vector Graphics',
 'svgz': 'Scalable Vector Graphics',
 'tif': 'Tagged Image File Format',
 'tiff': 'Tagged Image File Format'}